In [ ]:
!python -m pip install semantic-kernel==1.0.4
!python -m pip install azure-core==1.30.1
!python -m pip install azure-search-documents==11.4.0

Load values from your .env file.  Rename the notebook.env to .env and add your values.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aiSerchEndpoint =os.getenv("AZURE_AI_SEARCH_ENDPOINT")
aiSearchKey = os.getenv("AZURE_AI_SEARCH_KEY")


In [2]:
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai.services.azure_text_embedding import AzureTextEmbedding
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.kernel import Kernel
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
kernel = Kernel()

service_id = "manuallookup"


azure_chat_service = AzureChatCompletion(
                        service_id=service_id,
                        deployment_name=deployment,
                        endpoint=endpoint,
                        api_key=api_key,
                    )

embedding_gen = AzureTextEmbedding(service_id="embedding", deployment_name=embedding_deployment)
kernel.add_service(azure_chat_service)
kernel.add_service(embedding_gen)



In [3]:
from semantic_kernel.connectors.memory.azure_cognitive_search import AzureCognitiveSearchMemoryStore

acs_memory_store = AzureCognitiveSearchMemoryStore( vector_size=1536, search_endpoint=aiSerchEndpoint , admin_key=aiSearchKey)

memory = SemanticTextMemory(storage=acs_memory_store, embeddings_generator=embedding_gen)
aiSearchPlugin = kernel.add_plugin(TextMemoryPlugin(memory), plugin_name="AISearch")
aiSearch = aiSearchPlugin["recall"]


kmmanuals is the index within AI Search created by the Kernel Memory Service.
>Documents are the french manuals from the user story
    

In [ ]:
result = await aiSearch.invoke(kernel, ask="How do I prepare the RRU", collection='kmmanuals')
print(f"Answer: {result[0].text}\n")

In [9]:
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

prompt = """
                          You are an AI assistant that helps machine operators ask questions from manuals.  
                            The questions could be in English or French, and the documents are currently in French. 
                            Find the answer to the question.
                            TRANSLATE THE ANSWER TO THE LANGUAGE THE QUESTION IS ASKED

                            ###
                            How can I replace the silicon hose?

                            To change the silicon hose follow these steps:
                            1. Remove the hose
                            2. replace the hose
                            3.tighten the hose
                            4. restart the machine

                          Question: {{$input}}
                          Tool call result: {{AISearch.recall $input collection='kmmanuals'}}
                          If the answer is empty say "I don't know", otherwise reply with a preview of the answer.
                          """
# AISearch.recall $input collection='machinestest' relevance=0 limit=1

        
execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="operator",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The question asked", is_required=True),
        #InputVariable(name="index", description="The index of AI Search", is_required=True)
    ],
    execution_settings=execution_settings
)


op_query = kernel.add_function(
    prompt=prompt,
    plugin_name="OperatorsPlugIn",
    function_name="OperatorsQuery",
    prompt_template_config=prompt_template_config
)



In [10]:
question = "How do I prepare the RRU"
print(f"Question: {question}")
result = await op_query.invoke(kernel, input=question)
print(result)

Question: How do I prepare the RRU


Error invoking function `AISearch.recall`
Error rendering code block: Error invoking function `AISearch.recall`


TemplateRenderException: Error rendering code block: Error invoking function `AISearch.recall`